##Silver Data Transformations

In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *

In [0]:
df = spark.read.format("delta")\
        .option("header",True)\
        .option("inferSchema",True)\
        .load("abfss://bronze@netflixprojectdatalake.dfs.core.windows.net/netflix_titles")

In [0]:
df.limit(10).display()

In [0]:
df = df.fillna({"duration_minutes" : 0, "duration_seasons":1})

In [0]:
df.limit(10).display()

In [0]:
df = df.withColumn("duration_minutes",col('duration_minutes').cast(IntegerType()))\
            .withColumn("duration_seasons",col('duration_seasons').cast(IntegerType()))

In [0]:
df.printSchema()

In [0]:
df = df.withColumn("Shorttitle",split(col('title'),':')[0])
df.display()

In [0]:
df = df.withColumn("rating",split(col('rating'),'-')[0])
df.limit(5).display()

In [0]:
df = df.withColumn("type_flag",when(col('type')=='Movie',1)\
                        .when(col('type')=='TV Show',2)\
                        .otherwise(0))
df.limit(5).display()

In [0]:
from pyspark.sql.window import Window

In [0]:
df = df.withColumn("duration_ranking",dense_rank().over(Window.orderBy(col('duration_minutes').desc())))

In [0]:
df.limit(5).display()

In [0]:
df.createOrReplaceTempView("temp_view")
df.createOrReplaceGlobalTempView("global_view")

In [0]:
df = spark.sql(""" 
                select * from global_temp.global_view
               
               """)

In [0]:
df.limit(5).display()

In [0]:
df_vis = df.groupBy("type").agg(count("*").alias("total_count"))
display(df_vis)

Databricks visualization. Run in Databricks to view.

In [0]:
df.write.format("delta")\
        .mode("overwrite")\
        .option("path","abfss://silver@netflixprojectdatalake.dfs.core.windows.net/netflix_titles")\
        .save()